In [2]:
from multiprocessing import Process, Manager
import time
import itertools
import numpy as np
import gzip
import multiprocessing
def do_work(in_queue, out_list):
    while True:
        item = in_queue.get()
        line_no, line = item
        #print(type(line))
        # exit signal 
        if line == None:
            return
        
        # fake work
        #time.sleep(.5)
        #result = (line_no, line)
        #print(result[0])
        #out_list.append(result)
        out_list.append([line_no, np.array(line.split(b"\t")[1:]).astype(np.float16)])
        if line_no % 1000 == 0:
            print(line_no)


In [3]:
if __name__ == "__main__":
    expr_mat = np.zeros((16774, 235121), dtype = np.float16)

    num_workers = multiprocessing.cpu_count()//2

    manager = Manager()
    results = manager.list()
    work = manager.Queue(num_workers)

    # start for workers    
    pool = []
    for i in range(num_workers):
        p = Process(target=do_work, args=(work, results))
        p.start()
        pool.append(p)

    # produce data
    with gzip.open('Organoid_UCSC/exprMatrix.tsv.gz') as f:
        iters = itertools.chain(f, (None,)*num_workers)
        #counter = 0
        for num_and_line in enumerate(iters):
            work.put(num_and_line)
            #counter += 1
            #if counter == 1000:
            #    break
    for p in pool:
        p.join()

    # get the results
    # example:  [(1, "foo"), (10, "bar"), (0, "start")]
    #print sorted(results)

Process Process-2:
Traceback (most recent call last):
  File "/home/sban/miniconda3/envs/organoid/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/sban/miniconda3/envs/organoid/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4397d6d46376>", line 21, in do_work
    out_list.append([line_no, np.array(line.split(b"\t")[1:]).astype(np.float16)])
ValueError: could not convert string to float: 'WTC10SWeek10_TTCTTAGGTGTGCGTC'


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


In [4]:
len(results)

16774

In [8]:
results[0][0]

2

In [11]:
for i in range(len(results)):
    expr_mat[results[i][0]] = results[i][1]
    if i % 1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


IndexError: index 16774 is out of bounds for axis 0 with size 16774

In [12]:
np.sum(expr_mat != 0)

310711409

In [13]:
expr_mat.size

3943919654

In [14]:
310711409/3943919654

0.07878238814649037

In [4]:
expr_mat.nbytes

7887839308